# Summary Stats on a Proposed Rebalance

In [27]:
import json
import pandas as pd
from glob import glob

paths = glob("../data/*")

In [45]:
def make_in_out_apr_df(state_of_destinations) -> pd.DataFrame:
    df = pd.DataFrame()
    df["name"] = [dest["name"] for dest in state_of_destinations["destStates"]]
    df["totalAprIn"] = [dest["totalAprIn"] for dest in state_of_destinations["destStates"]]
    df["totalAprOut"] = [dest["totalAprOut"] for dest in state_of_destinations["destStates"]]
    df["destination"] = [dest["address"] for dest in state_of_destinations["destStates"]]
    df["lp_safePrice"] = [dest["spotPrice"] for dest in state_of_destinations["destStates"]]
    df["lp_spotPrice"] = [dest["safePrice"] for dest in state_of_destinations["destStates"]]
    return df


def build_summary_data(path: str) -> dict:

    with open(path, "r") as fin:
        data = json.load(fin)

    state_of_destinations = data.pop("sod")
    apr_df = make_in_out_apr_df(state_of_destinations)  # does not dilute

    destinationOut_name = apr_df[apr_df["destination"] == data["destinationOut"]]["name"].values[0]
    destinationIn_name = apr_df[apr_df["destination"] == data["destinationIn"]]["name"].values[0]

    totalAprIn = float(
        apr_df[apr_df["destination"] == data["destinationIn"]]["totalAprIn"].values[0] * 100
    )  # make sure that the dilution is accounted for here.
    totalAprOut = float(apr_df[apr_df["destination"] == data["destinationOut"]]["totalAprOut"].values[0] * 100)

    amountOutETH = int(data["amountOutETH"]) / 1e18
    minAmountInETH = int(data["minAmountIn"]) / 1e18

    expectedAPRIncrease = totalAprIn - totalAprOut
    slippage_percent = 100 - ((100 * int(data["minAmountInETH"])) / int(data["amountOutETH"]))

    swap_cost_eth = amountOutETH - minAmountInETH

    predicted_annualized_gain = ((totalAprIn / 100) * minAmountInETH) - ((totalAprOut / 100) * amountOutETH)

    # double check that this number is right.
    min_break_even_days = (swap_cost_eth * 365) / predicted_annualized_gain

    proposedRebalanceImpacts = {
        "date": pd.to_datetime(data["timestamp"], unit="s"),
        "destinationOut_name": destinationOut_name[15:],
        "destinationIn_name": destinationIn_name[15:],
        "totalAprOut": totalAprOut,
        "totalAprIn": totalAprIn,
        "amountOutETH": amountOutETH,
        "minAmountInETH": minAmountInETH,
        "swap_cost_eth": swap_cost_eth,
        "expectedAPRIncrease": expectedAPRIncrease,
        "slippagePercent": slippage_percent,
        "predicted_annualized_gain": predicted_annualized_gain,
        "min_break_even_days": min_break_even_days,
    }
    return proposedRebalanceImpacts


rebalance_df = pd.DataFrame.from_records(build_summary_data(path) for path in paths)
rebalance_df = rebalance_df.set_index("date")
rebalance_df.sort_index()

,destinationOut_name,destinationIn_name,totalAprOut,totalAprIn,amountOutETH,minAmountInETH,swap_cost_eth,expectedAPRIncrease,slippagePercent,predicted_annualized_gain,min_break_even_days
date,,,,,,,,,,,
2024-07-12 23:20:47,Ether-osETH/rETH,Ether-Balancer ETHx/wstETH,6.847850,10.772455,8.912144,8.804792,0.107352,3.924605,0.462451,0.338202,115.858688
2024-07-29 21:08:19,Ether-osETH/rETH,Ether-Curve.fi Factory Pool: ETHx-ETH,6.808693,11.861191,35.662150,34.759751,0.902399,5.052498,0.276671,1.694794,194.345507
2024-07-30 04:37:06,Ether-osETH/rETH,Ether-Curve.fi Factory Pool: ETHx-ETH,6.801833,11.814118,35.662153,34.753961,0.908192,5.012286,0.293599,1.680194,197.292808
2024-07-30 04:55:08,Ether-osETH/rETH,Ether-Curve.fi Factory Pool: ETHx-ETH,6.801833,11.814118,35.662153,34.753970,0.908183,5.012286,0.293573,1.680195,197.290723
2024-07-30 23:29:57,Ether-osETH/rETH,Ether-Curve.fi Factory Pool: ETHx-ETH,6.302030,12.031603,35.659201,34.752917,0.906284,5.729573,0.278808,1.934079,171.034139
2024-07-30 23:41:38,Ether-osETH/rETH,Ether-Curve.fi Factory Pool: ETHx-ETH,6.302030,12.031603,35.659201,34.800815,0.858386,5.729573,0.141370,1.939842,161.513577
2024-07-31 20:45:09,Ether-osETH/rETH,Ether-Curve.fi Factory Pool: ETHx-ETH,6.297411,11.647200,35.660445,34.792283,0.868162,5.349789,0.162350,1.806642,175.396655
2024-07-31 21:11:41,Ether-osETH/rETH,Ether-Curve.fi Factory Pool: ETHx-ETH,6.297411,11.647200,35.660445,34.793837,0.866608,5.349789,0.157891,1.806823,175.065180
2024-07-31 21:54:06,Ether-osETH/rETH,Ether-Curve.fi Factory Pool: ETHx-ETH,6.297411,11.647200,35.660445,34.794553,0.865891,5.349789,0.155837,1.806906,174.912401


None of these rebalances would pass because the days to break even are always much greater than the swap cost off set period. Actual execution is much better. 

In [46]:
data

{'timestamp': 1722382898,
 'sodOnly': False,
 'chainId': '1',
 'solverAddress': '0x2C26808b567BA224652f4eB20D45df4bccC29470',
 'poolAddress': '0x49C4719EaCc746b87703F964F09C22751F397BA0',
 'destinationOut': '0x772C047f317381c8F2DBd7B43E13B704EfFdDD45',
 'tokenOut': '0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d',
 'amountOut': '34789300355926847071',
 'amountOutETH': '35659200528416075776',
 'destinationIn': '0x2E5A8C3aE475734Ece6443B5E68F7fA63133AF3D',
 'tokenIn': '0x59Ab5a5b5d617E478a2479B0cAD80DA7e2831492',
 'minAmountIn': '34800814573470052352',
 'minAmountInETH': '35608789063513743360',
 'steps': [{'stepType': 'removeLiquidity',
   'poolType': 'curveV1',
   'poolAddress': '0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d',
   'lpTokenAddress': '0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d',
   'lpAmountOut': '34789300355926847071',
   'tokens': ['0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38',
    '0xae78736Cd615f374D3085123A210448E74Fc6393'],
   'minTokenAmounts': ['20371313657141530624', '132

In [ ]:
# look at the specific route faster

In [ ]:
 'amountOutETH': '35659200528416075776',
 'destinationIn': '0x2E5A8C3aE475734Ece6443B5E68F7fA63133AF3D',
 'tokenIn': '0x59Ab5a5b5d617E478a2479B0cAD80DA7e2831492',
 'minAmountIn': '34800814573470052352',
 'minAmountInETH': '35608789063513743360',

In [44]:
35608789063513743360 / 35659200528416075776

0.9985862985104739

In [29]:
with open(paths[2], "r") as fin:
    data = json.load(fin)


token_symbols = {
    "0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE": "ETH",
    "0xA35b1B31Ce002FBF2058D22F30f95D405200A15b": "ETHx",
    "0xae78736Cd615f374D3085123A210448E74Fc6393": "rETH",
    "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2": "wETH",
    "0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38": "osETH",
    "0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0": "wstETH",
}


def _address_to_name(addr: str):
    return token_symbols[addr]  # stub, use on-chain calls when needed


def _handle_lifi_swap(step: dict):
    payload = step["payload"]

    tokenFrom = _address_to_name(payload["action"]["fromToken"]["address"])
    tokenTo = _address_to_name(payload["action"]["toToken"]["address"])

    estimate = payload["estimate"]
    toAmount = int(estimate["toAmount"])
    fromAmount = int(estimate["fromAmount"])

    return {
        "dex": step["dex"],
        "type": "swap",
        "tokenBought": tokenTo,
        "tokenSold": tokenFrom,
        "amountBought": toAmount,
        "amountSold": fromAmount,
        "balance_changes": {tokenTo: toAmount / 1e18, tokenFrom: -fromAmount / 1e18},
    }


def _handle_0x_swap(step: dict):
    return {
        "dex": step["dex"],
        "type": "swap",
        "tokenBought": _address_to_name(step["tokenIn"]),
        "tokenSold": _address_to_name(step["tokenOut"]),
        "amountBought": int(step["payload"]["grossBuyAmount"]),
        "amountSold": int(step["payload"]["grossSellAmount"]),
        "balance_changes": {
            _address_to_name(step["tokenIn"]): int(step["payload"]["grossBuyAmount"]) / 1e18,
            _address_to_name(step["tokenOut"]): -int(step["payload"]["grossSellAmount"]) / 1e18,
        },
    }


def _handle_wrap(step: dict):
    step["balance_changes"] = {
        "wETH": int(step["amountOut"]),
        "ETH": -int(step["amountIn"]),
    }  # values are the same
    return step


def _handle_unwrap(step: dict):
    step["balance_changes"] = {
        "wETH": -int(step["amountOut"]) / 1e18,
        "ETH": int(step["amountIn"]) / 1e18,
    }
    return step


def _handle_add_liqudity(step: dict):
    token0Out, token1Out = step["tokens"]
    token0_amount, token1_amount = step["amounts"]
    # add lp token amounts
    return {
        "type": step["stepType"],
        "token0Out": _address_to_name(token0Out),
        "token0_amount": token0_amount,
        "token1Out": _address_to_name(token1Out),
        "token1_amount": token1_amount,
        "balance_changes": {
            _address_to_name(token0Out): -int(token0_amount) / 1e18,
            _address_to_name(token1Out): -int(token1_amount) / 1e18,
        },
    }


def _handle_remove_liqudity(step: dict):
    token0Out, token1Out = step["tokens"]
    token0_amount, token1_amount = step["minTokenAmounts"]
    # add lp tokens amounts?

    return {
        "type": step["stepType"],
        "token0Out": _address_to_name(token0Out),
        "token0_amount": int(token0_amount),
        "token1Out": _address_to_name(token1Out),
        "token1_amount": int(token1_amount),
        "balance_changes": {
            _address_to_name(token0Out): int(token0_amount) / 1e18,
            _address_to_name(token1Out): int(token1_amount) / 1e18,
        },
    }


short_details = []
for step in data["steps"]:
    if step["stepType"] == "swap":
        if step["dex"] == "0x":
            details = _handle_0x_swap(step)
        elif step["dex"] == "lifi":
            details = _handle_lifi_swap(step)
        else:
            print(step)
            raise ValueError("bad path")

    elif step["stepType"] == "removeLiquidity":
        details = _handle_remove_liqudity(step)

    elif step["stepType"] == "addLiquidity":
        details = _handle_add_liqudity(step)

    elif step["stepType"] == "unwrap":
        details = _handle_unwrap(step)

    elif step["stepType"] == "wrap":
        details = _handle_wrap(step)

    else:
        print(step)
        raise ValueError("bad path")

    short_details.append(details)

changes = [b["balance_changes"] for b in short_details]
changes

[{'osETH': 20.37131365714153, 'rETH': 13.218964856233214},
 {'ETHx': 20.15252579706275, 'osETH': -20.37131365714153},
 {'wETH': 11.98344949525336, 'rETH': -10.714937400063144},
 {'wETH': -11.970867472455819, 'ETH': 11.970867472455819},
 {'ETHx': 2.7035192630543996, 'rETH': -2.50402745617007},
 {'ETHx': -22.83204735560628, 'ETH': -11.970867472455819}]

In [39]:
data["steps"][1]

{'stepType': 'swap',
 'dex': 'lifi',
 'tokenOut': '0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38',
 'amountOut': '20371313657141530624',
 'tokenIn': '0xA35b1B31Ce002FBF2058D22F30f95D405200A15b',
 'minAmountIn': '0',
 'payload': {'type': 'lifi',
  'id': '39abf824-aeb8-4c66-ac28-4710d6318621:0',
  'tool': '1inch',
  'toolDetails': {'key': '1inch',
   'name': '1inch',
   'logoURI': 'https://raw.githubusercontent.com/lifinance/types/main/src/assets/icons/exchanges/oneinch.png'},
  'action': {'fromToken': {'address': '0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38',
    'chainId': 1,
    'symbol': 'osETH',
    'decimals': 18,
    'name': 'Staked ETH',
    'coinKey': 'osETH',
    'logoURI': 'https://static.debank.com/image/eth_token/logo_url/0xf1c9acdc66974dfb6decb12aa385b9cd01190e38/4b9f533a91f012c8b142dd3b0755adae.png',
    'priceUSD': '3356.4333504428027'},
   'fromAmount': '20371313657141530624',
   'toToken': {'address': '0xA35b1B31Ce002FBF2058D22F30f95D405200A15b',
    'chainId': 1,
    'symb

In [42]:
data

{'timestamp': 1722382898,
 'sodOnly': False,
 'chainId': '1',
 'solverAddress': '0x2C26808b567BA224652f4eB20D45df4bccC29470',
 'poolAddress': '0x49C4719EaCc746b87703F964F09C22751F397BA0',
 'destinationOut': '0x772C047f317381c8F2DBd7B43E13B704EfFdDD45',
 'tokenOut': '0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d',
 'amountOut': '34789300355926847071',
 'amountOutETH': '35659200528416075776',
 'destinationIn': '0x2E5A8C3aE475734Ece6443B5E68F7fA63133AF3D',
 'tokenIn': '0x59Ab5a5b5d617E478a2479B0cAD80DA7e2831492',
 'minAmountIn': '34800814573470052352',
 'minAmountInETH': '35608789063513743360',
 'steps': [{'stepType': 'removeLiquidity',
   'poolType': 'curveV1',
   'poolAddress': '0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d',
   'lpTokenAddress': '0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d',
   'lpAmountOut': '34789300355926847071',
   'tokens': ['0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38',
    '0xae78736Cd615f374D3085123A210448E74Fc6393'],
   'minTokenAmounts': ['20371313657141530624', '132

In [41]:
data["steps"][2]

{'stepType': 'swap',
 'dex': 'lifi',
 'tokenOut': '0xae78736Cd615f374D3085123A210448E74Fc6393',
 'amountOut': '10714937400063143936',
 'tokenIn': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
 'minAmountIn': '0',
 'payload': {'type': 'lifi',
  'id': '9b368a9c-624f-49a8-ac37-eeb533823584:0',
  'tool': 'lifidexaggregator',
  'toolDetails': {'key': 'lifidexaggregator',
   'name': 'LI.FI DEX Aggregator',
   'logoURI': 'https://raw.githubusercontent.com/lifinance/types/main/src/assets/icons/exchanges/lifidexaggregator.svg'},
  'action': {'fromToken': {'address': '0xae78736Cd615f374D3085123A210448E74Fc6393',
    'chainId': 1,
    'symbol': 'rETH',
    'decimals': 18,
    'name': 'Rocket Pool ETH',
    'coinKey': 'rETH',
    'logoURI': 'https://static.debank.com/image/eth_token/logo_url/0xae78736cd615f374d3085123a210448e74fc6393/0a56aa87c04449332f88702b2bd5f45c.png',
    'priceUSD': '3648.5909727942458'},
   'fromAmount': '10714937400063143936',
   'toToken': {'address': '0xC02aaA39b223FE8D0A

In [32]:
df = pd.DataFrame.from_records(changes).fillna(0)
df

,osETH,rETH,ETHx,wETH,ETH
0,20.371314,13.218965,0.000000,0.000000,0.000000
1,-20.371314,0.000000,20.152526,0.000000,0.000000
2,0.000000,-10.714937,0.000000,11.983449,0.000000
3,0.000000,0.000000,0.000000,-11.970867,11.970867
4,0.000000,-2.504027,2.703519,0.000000,0.000000
5,0.000000,0.000000,-22.832047,0.000000,-11.970867


In [37]:
df.cumsum().round(4)

,osETH,rETH,ETHx,wETH,ETH
0,20.3713,13.219,0.0000,0.0000,0.0000
1,0.0000,13.219,20.1525,0.0000,0.0000
2,0.0000,2.504,20.1525,11.9834,0.0000
3,0.0000,2.504,20.1525,0.0126,11.9709
4,0.0000,-0.000,22.8560,0.0126,11.9709
5,0.0000,-0.000,0.0240,0.0126,0.0000


In [40]:
data["steps"][-1]

{'stepType': 'addLiquidity',
 'poolType': 'curveV1',
 'poolAddress': '0x59Ab5a5b5d617E478a2479B0cAD80DA7e2831492',
 'lpTokenAddress': '0x59Ab5a5b5d617E478a2479B0cAD80DA7e2831492',
 'tokens': ['0xA35b1B31Ce002FBF2058D22F30f95D405200A15b',
  '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'],
 'amounts': ['22832047355606281728', '11970867472455819264'],
 'minLpAmountOut': '0'}

### Questions
- is unwrap always for WETH or can it be for wstETH as well.
- Can we run this in the past?
- Other dexes than 0x and Lifi? need a custom way to process the jsons created by this index.
- the lefover is the solver margin right?


##




In [47]:
# more high level, look at the route later